<a href="https://colab.research.google.com/github/RochaErik/AlgorithmComparison/blob/main/AlgorithmComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [307]:
!pip install catboost
!pip install lightgbm
!pip install xgboost

In [308]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# **Wine Dataset**

In [309]:
wine_df = pd.read_csv('/content/drive/MyDrive/DatasetSeminario/Wine/wine.data', header=None)

In [310]:
wine_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [311]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       178 non-null    int64  
 1   1       178 non-null    float64
 2   2       178 non-null    float64
 3   3       178 non-null    float64
 4   4       178 non-null    float64
 5   5       178 non-null    int64  
 6   6       178 non-null    float64
 7   7       178 non-null    float64
 8   8       178 non-null    float64
 9   9       178 non-null    float64
 10  10      178 non-null    float64
 11  11      178 non-null    float64
 12  12      178 non-null    float64
 13  13      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [312]:
wine_df.nunique()

0       3
1     126
2     133
3      79
4      63
5      53
6      97
7     132
8      39
9     101
10    132
11     78
12    122
13    121
dtype: int64

In [313]:
wine_df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [314]:
X = wine_df.iloc[:, 1:]
y = wine_df.iloc[:, 0]

In [315]:
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [316]:
X.shape

(178, 13)

In [317]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: 0, dtype: int64

In [318]:
y.shape

(178,)

In [319]:
# Since version 1.3.2 XGBoost needs target columns to start with 0 value

le = LabelEncoder()
y = le.fit_transform(y)

In [320]:
names = [
          'AdaBoost',
          'CatBoost',
          'LightGBM',
          'XGBoost'
        ]

classifiers = [
                AdaBoostClassifier(),
                CatBoostClassifier(silent=True),
                LGBMClassifier(verbosity=-1),
                XGBClassifier()
              ]

In [321]:
rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)

In [322]:
# wine_scores_mean = []
# wine_scores_std = []

# for name, clf in zip(names, classifiers):
#   results = cross_val_score(clf, X, y, cv=rkf)
#   wine_scores_mean.append(results.mean()*100)
#   wine_scores_std.append(results.std()*100)
#   print(f'--------- {name} on Wine Dataset ---------')
#   print(results)
#   print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
#   print('------------------------------')

In [323]:
# wine_scores_mean

In [324]:
# wine_scores_std

In [325]:
# Algo_results = pd.DataFrame()
# Algo_results['Name'] = names
# Algo_results['Results'] = wine_scores_mean

In [326]:
# Algo_results

# **Breast Cancer Dataset**

In [327]:
breast_cancer_df = pd.read_csv('/content/drive/MyDrive/DatasetSeminario/BreastCancer/breastCancer.data', header=None, na_values='?')

In [328]:
breast_cancer_df.head()

,0,1,2,3,4,5,6,7,8,9
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [329]:
print(breast_cancer_df.shape)

(286, 10)


In [330]:
breast_cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       286 non-null    object
 1   1       286 non-null    object
 2   2       286 non-null    object
 3   3       286 non-null    object
 4   4       286 non-null    object
 5   5       278 non-null    object
 6   6       286 non-null    int64 
 7   7       286 non-null    object
 8   8       285 non-null    object
 9   9       286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [331]:
# breast_cancer_df[1] = breast_cancer_df[1].astype('string')
# breast_cancer_df[3] = breast_cancer_df[3].astype('string')
# breast_cancer_df[4] = breast_cancer_df[4].astype('string')

In [338]:
breast_cancer_df.nunique()

0     2
1     6
2     3
3    11
4     7
5     2
6     3
7     2
8     5
9     2
dtype: int64

In [339]:
breast_cancer_df.isna().sum()

0    0
1    0
2    0
3    0
4    0
5    8
6    0
7    0
8    1
9    0
dtype: int64

In [340]:
# Checking the proportion of NaN values

missing_count = breast_cancer_df[5].isna().sum()
total_count = len(breast_cancer_df[5])
missing_ratio = missing_count / total_count
missing_ratio

0.027972027972027972

In [341]:
# The proportion of NaN values was low so we opt to remove the rows

breast_cancer_df.dropna(inplace=True)

In [342]:
breast_cancer_df.isna().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [332]:
age_dummies = {
                '10-19': 0,
                '20-29': 1,
                '30-39': 2,
                '40-49': 3,
                '50-59': 4,
                '60-69': 5,
                '70-79': 6,
                '80-89': 7,
                '90-99': 8
              }
breast_cancer_df[1].replace(age_dummies, inplace=True)

In [333]:
breast_cancer_df[1]

0      2
1      3
2      3
3      5
4      3
      ..
281    2
282    2
283    5
284    3
285    4
Name: 1, Length: 286, dtype: int64

In [334]:
size_dummies = {
                '0-4': 0,
                '5-9': 1,
                '10-14': 2,
                '15-19': 3,
                '20-24': 4,
                '25-29': 5,
                '30-34': 6,
                '35-39': 7,
                '40-44': 8,
                '45-49': 9,
                '50-54': 10,
                '55-59': 11
                }
breast_cancer_df[3].replace(size_dummies, inplace=True)

In [335]:
breast_cancer_df[3]

0      6
1      4
2      4
3      3
4      0
      ..
281    6
282    4
283    4
284    6
285    6
Name: 3, Length: 286, dtype: int64

In [336]:
invnodes_dummies = {
                '0-2': 0,
                '3-5': 1,
                '6-8': 2,
                '9-11': 3,
                '12-14': 4,
                '15-17': 5,
                '18-20': 6,
                '21-23': 7,
                '24-26': 8,
                '27-29': 9,
                '30-32': 10,
                '33-35': 11,
                '36-39': 12,
              }
breast_cancer_df[4].replace(invnodes_dummies, inplace=True)

In [337]:
breast_cancer_df[4]

0      0
1      0
2      0
3      0
4      0
      ..
281    0
282    0
283    0
284    1
285    1
Name: 4, Length: 286, dtype: int64

In [343]:
X = breast_cancer_df.iloc[:, 1:]
y = breast_cancer_df.iloc[:, 0]

In [344]:
X.head()

,1,2,3,4,5,6,7,8,9
0,2,premeno,6,0,no,3,left,left_low,no
1,3,premeno,4,0,no,2,right,right_up,no
2,3,premeno,4,0,no,2,left,left_low,no
3,5,ge40,3,0,no,2,right,left_up,no
4,3,premeno,0,0,no,2,right,right_low,no


In [346]:
y.head()

0    no-recurrence-events
1    no-recurrence-events
2    no-recurrence-events
3    no-recurrence-events
4    no-recurrence-events
Name: 0, dtype: object

In [347]:
# Since version 1.3.2 XGBoost needs target columns to start with 0 value

le = LabelEncoder()
y = le.fit_transform(y)

In [348]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [349]:
breast_cancer_mean = []
breast_cancer_std = []

for name, clf in zip(names, classifiers):
  results = cross_val_score(clf, X, y, cv=rkf)
  breast_cancer_mean.append(results.mean()*100)
  breast_cancer_std.append(results.std()*100)
  print(f'--------- {name} on Breast Cancer Dataset ---------')
  print(results)
  print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
  print('------------------------------')

ValueError: ignored

In [ ]:
breast_cancer_mean

In [ ]:
breast_cancer_std